In [1]:
import torch
import math
import pandas as pd

# Usaremos float64 para melhor precisão numérica
torch.set_default_dtype(torch.float64)

def grad_autograd(f, x):
    """Retorna gradiente de f em x, mantendo grafo para segunda ordem."""
    x_ = x.clone().detach().requires_grad_(True)
    y = f(x_)
    if y.ndim != 0:
        raise ValueError("A função f deve retornar um escalar.")
    g, = torch.autograd.grad(y, x_, create_graph=True)
    return y, g, x_

def hessian_autograd(f, x):
    """Retorna valor, gradiente e Hessiana de f em x."""
    y, g, x_ = grad_autograd(f, x)
    H_rows = []
    for i in range(x_.numel()):
        row_i, = torch.autograd.grad(g[i], x_, retain_graph=True, create_graph=True)
        H_rows.append(row_i)
    H = torch.stack(H_rows)
    return y, g, H

In [2]:

def directional_derivative(f, x, v):
    """Derivada direcional de f em x na direção v (normalizada)."""
    v = v.clone().detach().to(x)
    v = v / v.norm()
    _, g, _ = grad_autograd(f, x)
    return torch.dot(g, v)

def df_grad_to_table(g, names):
    return pd.DataFrame({"variável": names, "∂f/∂variável": [float(gi) for gi in g.detach()]})

def df_hessian_to_table(H, names):
    return pd.DataFrame(H.detach().numpy(), index=names, columns=names)


# --- Exemplo 1: Polinômio em 3 variáveis ---
names1 = ["x","y","z"]
def f1(u):
    x, y, z = u
    return 2*x**3 - y**2 * z + 4*x*z - 5*y + 7

p1 = torch.tensor([1.0, -2.0, 0.5])  # ponto de avaliação
v1 = torch.tensor([1.0, 1.0, -1.0])  # direção qualquer

y1, g1, H1 = hessian_autograd(f1, p1)
lap1 = torch.trace(H1)
dir1 = directional_derivative(f1, p1, v1)

print("f(p1) =", float(y1))
display(df_grad_to_table(g1, names1))
display(df_hessian_to_table(H1, names1))
print("Laplaciano =", float(lap1))
print("Derivada direcional na direção v1 =", float(dir1))


In [5]:
# --- Exemplo 2: Trigonométrica mista em 2 variáveis ---
names2 = ["x","y"]
def f2(u):
    x, y = u
    return torch.sin(x)*torch.cos(y) + x**2 * y

p2 = torch.tensor([math.pi/3, math.pi/6])
v2 = torch.tensor([0.5, -1.0])

y2, g2, H2 = hessian_autograd(f2, p2)
lap2 = torch.trace(H2)
dir2 = directional_derivative(f2, p2, v2)

print("f(p2) =", float(y2))
display(df_grad_to_table(g2, names2))
display(df_hessian_to_table(H2, names2))
print("Laplaciano =", float(lap2))
print("Derivada direcional na direção v2 =", float(dir2))


f(p2) = 1.324190308894441


,variável,∂f/∂variável
0,x,1.529635
1,y,0.663610


,x,y
x,0.297198,1.844395
y,1.844395,-0.750000


Laplaciano = -0.45280244880340237
Derivada direcional na direção v2 = 0.09052291633407024
